In [1]:
ALL_CARD_TYPES = ["Wizard", "Ghost","Knight","Firespell","plant","skeleton"]
CARD_TO_INDEX = {name: i for i, name in enumerate(ALL_CARD_TYPES)}
def encode_cards(card_list):
    vec = [0] * len(ALL_CARD_TYPES)
    for c in card_list:
        if isinstance(c, str):  # already a name
            idx = CARD_TO_INDEX.get(c, None)
        else:  # card object
            idx = CARD_TO_INDEX.get(c.__class__.__name__, None)
        if idx is not None:
            vec[idx] = 1
    return vec
import numpy as np

def snapshot_to_vector(snapshot):
    numeric_features = [
        snapshot["My_Mana"],
        snapshot["Their_Mana"],
        snapshot["My_Health"],
        snapshot["Their_Health"]
    ]
    seen_vec = encode_cards(snapshot["Their_Card_Types_Seen"])
    visible_vec = encode_cards(snapshot["Their_Currently_Visible_Cards"])
    active_vec = encode_cards(snapshot["Active_Cards"])
    deck_vec = encode_cards(snapshot["Deck"])

    return np.array(numeric_features + seen_vec + visible_vec + active_vec + deck_vec, dtype=float)
import torch


encode_cards(["Ghost"])



[0, 1, 0, 0, 0, 0]

In [2]:

from abc import ABC, abstractmethod
from copy import deepcopy

# -----------------------------
# Card base class
# -----------------------------
class Card(ABC):
    def __init__(self, type_, team, mana_cost, name="", hidden=False):
        self.type = type_
        self.team = team
        self.mana_cost = mana_cost
        self.name = name
        self.hidden = hidden
        self.deployed = False
        self.dead = False
        self.square = None
        self.exhausted=False

    @abstractmethod
    def get_possible_moves(self, game, player):
        pass



# -----------------------------
# Square and BuffSquare
# -----------------------------
class Square:
    def __init__(self, index, troop=None):
        self.index = index
        self.troop = troop
        self.empty = troop is None

class BuffSquare:
    def __init__(self):
        self.empty = True

# -----------------------------
# Move base class
# -----------------------------
class Move(ABC):
    @abstractmethod
    def execute(self, game):
        pass

# -----------------------------
# DeployTroopMove
# -----------------------------
class DeployTroopMove(Move):
    def __init__(self, card, square):
        self.card = card
        self.square = square
        self.mana_cost = card.mana_cost

    def execute(self, game):
        player = game.players[self.card.team]
        if player.mana < self.mana_cost:
            if game.prnt:
                print("❌ Not enough mana to deploy!")
            return
        player.mana -= self.mana_cost
        self.square.troop = self.card
        self.square.empty = False
        self.card.deployed = True
        self.card.square = self.square
        if self.card in player.deck:
            player.deck.remove(self.card)
        player.active_cards.append(self.card)
        if game.prnt:
            print(f"🧙 {self.card.name} deployed on square {self.square.index}. Mana: {player.mana}")


class FireballMove(Move):
    def __init__(self, caster, target_square):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = 4

    def execute(self, game):
        player = self.caster.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                print("❌ Not enough mana for fireball!")
            return
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True

        if game.prnt:
            print(f"Player{player.team+1}'s Wizard on square {self.caster.square.index} casts Fireball at enemy square {idx}! Mana left: {player.mana}")

        splash_indices = [i for i in [idx-1, idx, idx+1] if 0 <= i < len(squares)]
        splash_damage = 6 / len(splash_indices)
        overkill = 0

        for i in splash_indices:
            sq = squares[i]
            if not sq.empty and sq.troop and hasattr(sq.troop, "health") and not sq.troop.dead:
                sq.troop.health -= splash_damage
                if game.prnt:
                    print(f"🔥 {splash_damage:.1f} dmg to troop on square {i} (remaining {sq.troop.health:.1f})")
                if sq.troop.health <= 0:
                    overkill += -sq.troop.health
                    if game.prnt:
                        print(f"💀 Troop on square {i} destroyed!")
                    sq.troop.die(opponent)
                    sq.troop = None
                    sq.empty = True
            else:
                overkill += splash_damage  # hitting empty square counts as overkill

        if overkill > 0:
            opponent.base_health -= overkill
            if game.prnt:
                print(f"🏰 {overkill:.1f} overkill damage to opponent base! Remaining: {opponent.base_health:.1f}")

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True
class HauntMove(Move):
    def __init__(self, caster, target_square):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = 1

    def execute(self, game):
        player = self.caster.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                pass
            return
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True

        if game.prnt:
            print(f"Player{player.team+1}'s Ghost on square {self.caster.square.index} casts Haunt at enemy square {idx}! Mana left: {player.mana}")

        
        if not self.target_square.empty:
            self.target_square.troop.health-=1
            if game.prnt:
                print(f" 1 damage to opponent at square {idx} Remaining: {self.target_square.troop.health:.1f}")
            if self.target_square.troop.health<=0:
                if game.prnt:
                    print(f" {self.target_square.troop.name} at square {idx} has died!")
                self.target_square.troop.die(opponent)
                self.target_square.troop = None
                self.target_square.empty = True
                
        
            
            
        

        
            
                
        opponent.base_health-=1
        if game.prnt:
            print(f" 1 damage to opponent base! Remaining: {opponent.base_health:.1f}")

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True

# -----------------------------
# EndTurnMove
# -----------------------------
class EndTurnMove(Move):
    def __init__(self, player):
        self.player = player
        

    def execute(self, game):
        # End the current player's turn first
        game.change_turn()
        game.moves_played += 1

        # Refresh the new player's deck and info
        new_player = game.players[game.turn]
        old_player = game.players[1 - game.turn]

        old_player.mana += 2
        old_player.refresh_deck()

        # Now the new player records information before acting
        new_player.invigorate()
        new_player.refresh_deck()
        new_player.record_information(game)

        if game.prnt:
            print(f"Player {old_player.team+1} ends turn. Mana: {old_player.mana}")

# -----------------------------
# WizardCard
# -----------------------------
class WizardCard(Card):
    def __init__(self, team, name):
        super().__init__(type_="troop", team=team, mana_cost=3, name="Wizard")
        self.health = 10
        self.hidden=False
        self.exhausted=False

    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= 4 and not self.exhausted:
                for sq in opponent.available_squares:
                    moves.append(FireballMove(self, sq))
        return moves
    def die(self, player):
        """Troop dies and returns to largedeck for reuse."""
        
        self.deployed = False
        if self in player.active_cards:
            player.active_cards.remove(self)
        if self in player.deck:
            player.deck.remove(self)
        player.largedeck.append(self)  # Recycle the card
        self.square = None
        self.health=10

# -----------------------------
# DecisionMaker (random for now)
# -----------------------------
class DecisionMaker:
    def __init__(self, difficulty, json_data=None):
        self.difficulty = difficulty
        self.json_data = json_data

    def choose_move(self, possible_moves,information):
        return np.random.choice(possible_moves)

# -----------------------------
# GamePlayer
# -----------------------------
class GamePlayer:
    def __init__(self, difficulty, game, deck, decision_maker):
        self.difficulty = difficulty
        self.game = game
        # Start with the full deck as "largedeck"
        self.largedeck = deepcopy(deck)
        self.deck = []  # current hand (max 5)
        self.active_cards = []
        self.decision_maker = decision_maker
        self.team = None
        self.available_squares = [Square(i) for i in range(4)]
        self.mana = 20
        self.base_health = 100
        # draw initial hand
        self.draw_cards()
        snapshot = {
           
            "My_Mana": self.mana,
            "Their_Mana": self.mana,
            "My_Health": self.base_health,
            "Their_Health": self.base_health,
            "Their_Card_Types_Seen": [],
            "Their_Currently_Visible_Cards": [],
            "Active_Cards":[],
            "Deck":[c.name for c in self.deck]
        }
        self.information=[snapshot_to_vector(snapshot)]
        self.known_enemy_types = set()
        self.passive_effectors=[]
        
    def clone(self):
        return deepcopy(self)

    def draw_cards(self):
        """Refill hand up to 5 cards from largedeck."""
        while len(self.deck) < 5 and self.largedeck:
            idx = np.random.choice(len(self.largedeck))
            self.deck.append(self.largedeck.pop(idx))

    def refresh_deck(self):
        """Call this at the start of a turn to ensure 5 cards in hand."""
        self.draw_cards()

    def get_possible_moves(self, game):
        moves = [EndTurnMove(self)]
        # Only consider cards that are alive
        extend_deck = [c for c in self.deck + self.active_cards if not getattr(c, "dead", False)]

        for card in extend_deck:
            if self.mana >= card.mana_cost:
                # Troops not yet deployed
                if card.type == "troop" and not card.deployed:
                    for sq in self.available_squares:
                        if sq.empty:
                            moves.append(DeployTroopMove(card, sq))
                # Spell cards
                elif card.type == "spell":
                    moves.extend(card.get_possible_moves(game, self))
                # Buff cards
                elif card.type == "buff" and getattr(self, "buff_square", BuffSquare()).empty:
                    moves.extend(card.get_possible_moves(game, self))
            # Already deployed troops can take moves
            if card.deployed:
                moves.extend(card.get_possible_moves(game, self))
        return moves

    def make_move(self, game):
        moves = self.get_possible_moves(game)
        if moves:
            move = self.decision_maker.choose_move(moves,torch.tensor(self.information, dtype=torch.float32))
            move.execute(game)
    

    def record_information(self, game):
        opponent = game.players[1 - self.team]
        visible_enemy_cards = [c for c in opponent.active_cards if not getattr(c, "hidden", False)]

        # update memory of seen types
        self.known_enemy_types.update(c.name for c in visible_enemy_cards)

        snapshot = {
           
            "My_Mana": self.mana,
            "Their_Mana": opponent.mana,
            "My_Health": self.base_health,
            "Their_Health": opponent.base_health,
            "Their_Card_Types_Seen": list(self.known_enemy_types),
            "Their_Currently_Visible_Cards": list({c.name for c in visible_enemy_cards}),
            "Active_Cards":[c.name for c in self.active_cards],
            "Deck":[c.name for c in self.deck]
        }
      
        self.information.append(snapshot_to_vector(snapshot))
    def invigorate(self):
        for card in self.active_cards:
            card.exhausted=False
# -----------------------------
# Game controller
# -----------------------------
class Game:
    def __init__(self, player_1, player_2, length=10, prnt=True, offset=[0,2]):
        self.turn = 0
        self.prnt = prnt
        self.moves_played = 0
        self.length = length
        self.game_over = False
        self.players = [player_1, player_2]
        player_1.game = self
        player_2.game = self
        player_1.team = 0
        player_2.team = 1
        player_2.base_health += offset[0]
        player_2.mana += offset[1]

    def change_turn(self):
        self.turn = 1 - self.turn

    def play(self):
        while self.moves_played < self.length and not self.game_over:
            player = self.players[self.turn]
            opponent = self.players[self.turn-1]
            if self.prnt:
                print(f"\n--- Turn {self.moves_played}, Player {self.turn+1} ---")
            player.make_move(self)
            
        return self.end_game()

    def end_game(self):
        p1, p2 = self.players
        if p1.base_health > p2.base_health:
            return 1
        elif p1.base_health < p2.base_health:
            return -1
        return 0

# -----------------------------
# Example simulation
# -----------------------------
def simulate(player1, player2, length=20, prnt=True, offset=[0,0]):
    p1 = player1.clone()
    p2 = player2.clone()
    game1 = Game(player1, player2, length, prnt, offset)
    game2 = Game(p1, p2, length, prnt, offset)
    return [game1.play(), -game2.play()]



In [3]:
# -----------------------------
# WizardCard
# -----------------------------
class GhostCard(Card):
    def __init__(self, team, name):
        super().__init__(type_="troop", team=team, mana_cost=3, name="Ghost")
        self.health = 10
        self.hidden=True
        self.exhausted=False
        self.ability_cost=1
    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= self.ability_cost and not self.exhausted:
                for sq in opponent.available_squares:
                    moves.append(HauntMove(self,sq))

        return moves
    def die(self, player):
        """Troop dies and returns to largedeck for reuse."""
        
        self.deployed = False
        if self in player.active_cards:
            player.active_cards.remove(self)
        if self in player.deck:
            player.deck.remove(self)
        player.largedeck.append(self)  # Recycle the card
        self.square = None
        self.health=10
        

In [4]:
# -----------------------------
results=[]
for a in range(20):
    # Create decks and players
    deck1 = [WizardCard(team=0,name="wizard1"),GhostCard(team=0,name="Ghost2"),WizardCard(team=0,name="wizard2")]
    deck2 = [WizardCard(team=1,name="wizard1"),GhostCard(team=1,name="Ghost2")]

    player1 = GamePlayer(difficulty=1, game=None, deck=deck1, decision_maker=DecisionMaker(1))
    player2 = GamePlayer(difficulty=1, game=None, deck=deck2, decision_maker=DecisionMaker(1))
    simulationresult = simulate(player1, player2,length=80,prnt=True,offset=[0,0])
    results.extend(simulationresult)# Run a test


--- Turn 0, Player 1 ---
🧙 Wizard deployed on square 1. Mana: 17

--- Turn 0, Player 1 ---
Player1's Wizard on square 1 casts Fireball at enemy square 2! Mana left: 13
🏰 6.0 overkill damage to opponent base! Remaining: 94.0

--- Turn 0, Player 1 ---
🧙 Ghost deployed on square 0. Mana: 10

--- Turn 0, Player 1 ---
Player1's Ghost on square 0 casts Haunt at enemy square 0! Mana left: 9
 1 damage to opponent base! Remaining: 93.0

--- Turn 0, Player 1 ---
Player 1 ends turn. Mana: 11

--- Turn 1, Player 2 ---
🧙 Wizard deployed on square 2. Mana: 17

--- Turn 1, Player 2 ---
Player2's Wizard on square 2 casts Fireball at enemy square 3! Mana left: 13
🏰 6.0 overkill damage to opponent base! Remaining: 94.0

--- Turn 1, Player 2 ---
🧙 Ghost deployed on square 0. Mana: 10

--- Turn 1, Player 2 ---
Player2's Ghost on square 0 casts Haunt at enemy square 3! Mana left: 9
 1 damage to opponent base! Remaining: 93.0

--- Turn 1, Player 2 ---
Player 2 ends turn. Mana: 11

--- Turn 2, Player 1 ---


C:\Users\allen\AppData\Local\Temp\ipykernel_3804\1183027248.py:319: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  move = self.decision_maker.choose_move(moves,torch.tensor(self.information, dtype=torch.float32))



Player1's Ghost on square 3 casts Haunt at enemy square 0! Mana left: 13
 1 damage to opponent base! Remaining: 99.0

--- Turn 0, Player 1 ---
Player1's Wizard on square 1 casts Fireball at enemy square 2! Mana left: 9
🏰 6.0 overkill damage to opponent base! Remaining: 93.0

--- Turn 0, Player 1 ---
Player 1 ends turn. Mana: 11

--- Turn 1, Player 2 ---
🧙 Ghost deployed on square 3. Mana: 17

--- Turn 1, Player 2 ---
Player2's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 16
 1 damage to opponent base! Remaining: 99.0

--- Turn 1, Player 2 ---
🧙 Wizard deployed on square 0. Mana: 13

--- Turn 1, Player 2 ---
Player2's Wizard on square 0 casts Fireball at enemy square 1! Mana left: 9
🔥 2.0 dmg to troop on square 1 (remaining 8.0)
🏰 4.0 overkill damage to opponent base! Remaining: 95.0

--- Turn 1, Player 2 ---
Player 2 ends turn. Mana: 11

--- Turn 2, Player 1 ---
Player1's Wizard on square 1 casts Fireball at enemy square 3! Mana left: 7
🔥 3.0 dmg to troop on square 3 (r

--- Turn 58, Player 1 ---
Player 1 ends turn. Mana: 4

--- Turn 59, Player 2 ---
Player 2 ends turn. Mana: 4

--- Turn 60, Player 1 ---
Player1's Ghost on square 1 casts Haunt at enemy square 3! Mana left: 3
 1 damage to opponent base! Remaining: 44.0

--- Turn 60, Player 1 ---
Player 1 ends turn. Mana: 5

--- Turn 61, Player 2 ---
Player2's Ghost on square 0 casts Haunt at enemy square 1! Mana left: 3
 1 damage to opponent at square 1 Remaining: 9.0
 1 damage to opponent base! Remaining: 41.0

--- Turn 61, Player 2 ---
Player 2 ends turn. Mana: 5

--- Turn 62, Player 1 ---
Player1's Ghost on square 1 casts Haunt at enemy square 2! Mana left: 4
 1 damage to opponent at square 2 Remaining: 9.0
 1 damage to opponent base! Remaining: 43.0

--- Turn 62, Player 1 ---
Player1's Wizard on square 2 casts Fireball at enemy square 1! Mana left: 0
🔥 2.0 dmg to troop on square 0 (remaining 1.0)
🔥 2.0 dmg to troop on square 2 (remaining 7.0)
🏰 2.0 overkill damage to opponent base! Remaining: 41.0



Player1's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 0
 1 damage to opponent base! Remaining: 66.0

--- Turn 30, Player 1 ---
Player 1 ends turn. Mana: 2

--- Turn 31, Player 2 ---
Player2's Wizard on square 0 casts Fireball at enemy square 1! Mana left: 0
🔥 2.0 dmg to troop on square 0 (remaining 3.0)
🏰 4.0 overkill damage to opponent base! Remaining: 67.0

--- Turn 31, Player 2 ---
Player 2 ends turn. Mana: 2

--- Turn 32, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 1
 1 damage to opponent base! Remaining: 65.0

--- Turn 32, Player 1 ---
Player 1 ends turn. Mana: 3

--- Turn 33, Player 2 ---
Player2's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 1
 1 damage to opponent base! Remaining: 66.0

--- Turn 33, Player 2 ---
Player 2 ends turn. Mana: 3

--- Turn 34, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 3! Mana left: 2
 1 damage to opponent at square 3 Remaining: 6.0
 1 damage to opponent ba

...

Player 2 ends turn. Mana: 2

--- Turn 64, Player 1 ---
Player 1 ends turn. Mana: 4

--- Turn 65, Player 2 ---
Player2's Ghost on square 1 casts Haunt at enemy square 0! Mana left: 1
 1 damage to opponent base! Remaining: 33.0

--- Turn 65, Player 2 ---
Player 2 ends turn. Mana: 3

--- Turn 66, Player 1 ---
🧙 Ghost deployed on square 1. Mana: 1

--- Turn 66, Player 1 ---
Player1's Ghost on square 1 casts Haunt at enemy square 1! Mana left: 0
 1 damage to opponent at square 1 Remaining: 0.0
 Ghost at square 1 has died!
 1 damage to opponent base! Remaining: 36.0

--- Turn 66, Player 1 ---
Player 1 ends turn. Mana: 2

--- Turn 67, Player 2 ---
🧙 Ghost deployed on square 2. Mana: 0

--- Turn 67, Player 2 ---
Player 2 ends turn. Mana: 2

--- Turn 68, Player 1 ---
Player 1 ends turn. Mana: 4

--- Turn 69, Player 2 ---
Player2's Ghost on square 2 casts Haunt at enemy square 2! Mana left: 1
 1 damage to opponent base! Remaining: 32.0

--- Turn 69, Player 2 ---
Player 2 ends turn. Mana: 3

--- 

Player1's Ghost on square 3 casts Haunt at enemy square 0! Mana left: 2
 1 damage to opponent base! Remaining: 41.0

--- Turn 62, Player 1 ---
Player 1 ends turn. Mana: 4

--- Turn 63, Player 2 ---
Player2's Ghost on square 1 casts Haunt at enemy square 3! Mana left: 2
 1 damage to opponent at square 3 Remaining: 9.0
 1 damage to opponent base! Remaining: 42.0

--- Turn 63, Player 2 ---
Player 2 ends turn. Mana: 4

--- Turn 64, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 3
 1 damage to opponent at square 2 Remaining: 2.0
 1 damage to opponent base! Remaining: 40.0

--- Turn 64, Player 1 ---
🧙 Wizard deployed on square 1. Mana: 0

--- Turn 64, Player 1 ---
Player 1 ends turn. Mana: 2

--- Turn 65, Player 2 ---
Player 2 ends turn. Mana: 6

--- Turn 66, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 1
 1 damage to opponent at square 2 Remaining: 1.0
 1 damage to opponent base! Remaining: 39.0

--- Turn 66, Player 1

Player 1 ends turn. Mana: 6

--- Turn 55, Player 2 ---
Player2's Wizard on square 0 casts Fireball at enemy square 2! Mana left: 1
🔥 2.0 dmg to troop on square 1 (remaining 0.0)
💀 Troop on square 1 destroyed!
🔥 2.0 dmg to troop on square 3 (remaining 3.0)
🏰 2.0 overkill damage to opponent base! Remaining: 44.0

--- Turn 55, Player 2 ---
Player 2 ends turn. Mana: 3

--- Turn 56, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 5
 1 damage to opponent base! Remaining: 43.0

--- Turn 56, Player 1 ---
🧙 Wizard deployed on square 0. Mana: 2

--- Turn 56, Player 1 ---
Player 1 ends turn. Mana: 4

--- Turn 57, Player 2 ---
Player2's Ghost on square 3 casts Haunt at enemy square 1! Mana left: 2
 1 damage to opponent base! Remaining: 43.0

--- Turn 57, Player 2 ---
Player 2 ends turn. Mana: 4

--- Turn 58, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 3! Mana left: 3
 1 damage to opponent at square 3 Remaining: 6.0
 1 damage to opponent b

Player 1 ends turn. Mana: 4

--- Turn 35, Player 2 ---
Player2's Ghost on square 2 casts Haunt at enemy square 2! Mana left: 1
 1 damage to opponent at square 2 Remaining: 0.0
 Wizard at square 2 has died!
 1 damage to opponent base! Remaining: 69.0

--- Turn 35, Player 2 ---
Player 2 ends turn. Mana: 3

--- Turn 36, Player 1 ---
🧙 Ghost deployed on square 0. Mana: 1

--- Turn 36, Player 1 ---
Player1's Ghost on square 0 casts Haunt at enemy square 2! Mana left: 0
 1 damage to opponent at square 2 Remaining: 6.0
 1 damage to opponent base! Remaining: 62.0

--- Turn 36, Player 1 ---
Player 1 ends turn. Mana: 2

--- Turn 37, Player 2 ---
Player2's Ghost on square 2 casts Haunt at enemy square 3! Mana left: 2
 1 damage to opponent base! Remaining: 68.0

--- Turn 37, Player 2 ---
Player 2 ends turn. Mana: 4

--- Turn 38, Player 1 ---
Player1's Ghost on square 0 casts Haunt at enemy square 3! Mana left: 1
 1 damage to opponent at square 3 Remaining: 1.0
 1 damage to opponent base! Remaining


--- Turn 25, Player 2 ---
Player 2 ends turn. Mana: 7

--- Turn 26, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 2
 1 damage to opponent base! Remaining: 65.0

--- Turn 26, Player 1 ---
Player 1 ends turn. Mana: 4

--- Turn 27, Player 2 ---
Player2's Ghost on square 3 casts Haunt at enemy square 0! Mana left: 6
 1 damage to opponent at square 0 Remaining: 2.0
 1 damage to opponent base! Remaining: 69.0

--- Turn 27, Player 2 ---
Player 2 ends turn. Mana: 8

--- Turn 28, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 3
 1 damage to opponent base! Remaining: 64.0

--- Turn 28, Player 1 ---
Player 1 ends turn. Mana: 5

--- Turn 29, Player 2 ---
Player2's Ghost on square 3 casts Haunt at enemy square 2! Mana left: 7
 1 damage to opponent base! Remaining: 68.0

--- Turn 29, Player 2 ---
Player2's Wizard on square 1 casts Fireball at enemy square 3! Mana left: 3
🔥 3.0 dmg to troop on square 3 (remaining 0.0)
💀 Troop o

In [5]:
from collections import Counter
print(Counter(results))

Counter({-1: 19, 1: 15, 0: 6})


In [6]:
player2.information

[array([ 20.,  20., 100., 100.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          1.,   1.,   0.,   0.,   0.,   0.]),
 array([ 20.,  22., 100., 100.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          1.,   1.,   0.,   0.,   0.,   0.]),
 array([11., 13., 95., 93.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.]),
 array([ 8., 10., 91., 89.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.]),
 array([ 5.,  7., 87., 84.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
         0.,  0.]),
 array([ 3.,  8., 86., 80.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.

In [7]:
np.shape(player2.information)

(41, 28)

In [8]:
torch.tensor(player2.information, dtype=torch.float32)

tensor([[ 20.,  20., 100.,  ...,   0.,   0.,   0.],
        [ 20.,  22., 100.,  ...,   0.,   0.,   0.],
        [ 11.,  13.,  95.,  ...,   0.,   0.,   0.],
        ...,
        [  4.,   3.,  26.,  ...,   0.,   0.,   0.],
        [  2.,   4.,  25.,  ...,   0.,   0.,   0.],
        [  3.,   2.,  24.,  ...,   0.,   0.,   0.]])